# EuroMillions

Fetch history, compute marginal frequencies, and score a few synthetic tickets.

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd

from euromillions import EuroMillionsGuess, evaluate_guess, load_history
from euromillions.get_draws import fetch_and_normalize


In [ ]:
# Fetch draws (uses cache if present) and persist to data/euromillions.csv
result = fetch_and_normalize(out_path=Path("data/euromillions.csv"), append=True)
df = result.dataframe
df.tail()


In [ ]:
# Marginal frequencies for balls and stars
ball_cols = [f"ball_{i}" for i in range(1, 6)]
star_cols = ["star_1", "star_2"]

ball_counts = pd.concat([df[c] for c in ball_cols]).value_counts().sort_index()
star_counts = pd.concat([df[c] for c in star_cols]).value_counts().sort_index()

fig, axes = plt.subplots(1, 2, figsize=(12, 3))
ball_counts.plot(kind="bar", ax=axes[0], color="#2f8dee", title="Ball frequency")
star_counts.plot(kind="bar", ax=axes[1], color="#f5a524", title="Star frequency")
plt.tight_layout()
plt.show()


In [ ]:
# Score a few synthetic tickets on the last 20 draws
guesses = [
    ("recency-biased", EuroMillionsGuess([1, 12, 23, 34, 45], [2, 10])),
    ("spread-out", EuroMillionsGuess([3, 15, 27, 36, 49], [4, 11])),
    ("low-numbers", EuroMillionsGuess([1, 2, 3, 4, 5], [1, 2])),
]

recent = df.tail(20)
scores = []
for name, guess in guesses:
    hits = [evaluate_guess(row, guess) for _, row in recent.iterrows()]
    ball_avg = sum(h[0] for h in hits) / len(hits)
    star_avg = sum(h[1] for h in hits) / len(hits)
    proto_score = ball_avg + 0.5 * star_avg
    scores.append({"name": name, "ball_avg": ball_avg, "star_avg": star_avg, "proto_score": proto_score})

pd.DataFrame(scores).sort_values("proto_score", ascending=False)
